# Brick optimisation larger unitaries

Created 19/08/2024

Objectives:
* Implement brick pattern optimisation strategy with wider unitaries (>2).
* Investigate performance as unitary width is widened.
* Create functions to automate.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from functools import reduce
from operator import mul

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/transverse_cluster_200_site_dmrg"

In [8]:
f_name = DATA_DIR + r"/0_90.h5"

In [9]:
f_name

'../data/transverse_cluster_200_site_dmrg/0_90.h5'

In [10]:
with h5py.File(f_name, 'r') as f:
    data = hdf5_io.load_from_hdf5(f)
    test_psi = data['wavefunction']

# Definitons

In [11]:
MAX_VIRTUAL_BOND_DIM = 8

In [12]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [13]:
npc_I = to_npc_array(np_I)
npc_X = to_npc_array(np_X)
npc_Y = to_npc_array(np_Y)
npc_Z = to_npc_array(np_Z)

In [14]:
test = SymmetryActionWithBoundaryUnitaries(
    test_psi,
    [np_X, np_I]*50
)

In [15]:
test.compute_svd_symmetry_action()

In [16]:
test.right_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

In [17]:
test.left_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vL', 'vL*']>

In [18]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [19]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Functions

### Leg and label functions

In [20]:
P_LEG_LABEL_REGEX_STRING = r"^p\d*$"
p_leg_pattern = re.compile(P_LEG_LABEL_REGEX_STRING)

def is_single_physical_leg_label(label):
    return bool(p_leg_pattern.match(label))

In [48]:
def is_grouped_physical_leg_label(leg_label):
    head, *body, tail = leg_label
    if (head != '(') or (tail != ')'):
        return False

    body = ''.join(body)
    legs = body.split('.')

    return all(is_single_physical_leg_label(l) for l in legs)

In [22]:
def is_physical_leg_label(label):
    out = (
        is_single_physical_leg_label(label)
        or is_grouped_physical_leg_label(label)
    )

    return out

In [23]:
def extract_single_physical_leg_label_from_tensor(b):
    out = next(
        l for l in b.get_leg_labels()
        if is_single_physical_leg_label(l)
    )

    return out

In [24]:
def conjugate_single_physical_leg_label(label):
    return label + '*'

In [25]:
def conjugate_grouped_physical_leg_label(label):
    _, *body, _ = label
    single_labels = ''.join(body).split('.')

    conjugate_labels = [
        conjugate_single_physical_leg_label(l)
        for l in single_labels
    ]

    conjugate_body = '.'.join(conjugate_labels)
    out = '(' + conjugate_body + ')'
    
    return out

In [26]:
def conjugate_leg_label(label):
    if is_single_physical_leg_label(label):
        return conjugate_single_physical_leg_label(label)
    elif is_grouped_physical_leg_label(label):
        return conjugate_grouped_physical_leg_label(label)
    else:
        raise ValueError

In [27]:
def get_physical_leg_labels(t):
    out = [
        ll for ll in t.get_leg_labels()
        if is_physical_leg_label(ll)
    ]

    return out

### Other

In [28]:
def contract_virtual_legs(tl, tr):
    return npc.tensordot(tl, tr, ['vR', 'vL'])

In [29]:
def combine_tensors(tensors):
    out = reduce(contract_virtual_legs, tensors)

    leg_labels = [
        extract_single_physical_leg_label_from_tensor(t)
        for t in tensors
    ]

    out = out.combine_legs(leg_labels)

    return out

In [30]:
def combine_b_tensors(b_tensors):
    renamed_tensors = [
        b.replace_label('p', f'p{i}')
        for i, b in enumerate(b_tensors)
    ]

    return combine_tensors(renamed_tensors)

In [31]:
def get_identity_operator(mps_tensor):
    p_leg_label = get_physical_leg_labels(mps_tensor)[0]
    p_leg = mps_tensor.get_leg(p_leg_label)
    p_leg_label_conj = conjugate_leg_label(p_leg_label)

    out = npc.diag(
        1,
        leg=p_leg,
        dtype='complex',
        labels=[p_leg_label, p_leg_label_conj]
    )

    return out

In [32]:
def svd_reduce_split_tensor(t, max_inner_dim=MAX_VIRTUAL_BOND_DIM, form='B'):
    # To-do: Add form conditions
    U, S, VH = npc.svd(t, compute_uv=True, inner_labels=['vR', 'vL'])

    # Update this code for other forms. Check tenpy docks.
    # Create U, V exponents and plug into for loop.
    for i in range(U.shape[1]):
        U[:, i] *= S[i]

    if max_inner_dim is not None:
        U = U[:, :max_inner_dim]
        VH = VH[:max_inner_dim, :]

    return U, VH

In [33]:
def split_combined_b(b, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = b.split_legs()

    num_sites = t.ndim - 2
    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    out_bs = list()

    for i, ll in enumerate(p_leg_labels[:-1]):
        tail_legs = p_leg_labels[(i+1):]
        t = t.combine_legs([['vL', ll], ['vR', *tail_legs]])

        U, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=MAX_VIRTUAL_BOND_DIM,
            form='B'
        )

        bl = U.split_legs().replace_label(ll, 'p')
        bl.itranspose(['vL', 'p', 'vR'])
        out_bs.append(bl)

        t = VH.split_legs()

    bl = t.replace_label(p_leg_labels[-1], 'p')
    bl.itranspose(['vL', 'p', 'vR'])
    out_bs.append(bl)

    return out_bs

In [83]:
def split_b(b, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    leg_label = get_physical_leg_labels(b)[0]

    if is_single_physical_leg_label(leg_label):
        return [b,]
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_b(b, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [34]:
def split_combined_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = u.split_legs()
    num_sites = t.ndim //2
    
    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    p_leg_labels_hc = [l+'*' for l in p_leg_labels]
    out_ws = list()

    for i, (ll, ll_hc) in enumerate(zip(p_leg_labels[:-1], p_leg_labels_hc)):
        tail_legs = p_leg_labels[(i+1):]
        tail_legs_hc = p_leg_labels_hc[(i+1):]

        left_leg_labels = [ll, ll_hc]
        if 'vL' in t.get_leg_labels():
            left_leg_labels.append('vL')
    
        t = t.combine_legs([left_leg_labels, [*tail_legs, *tail_legs_hc]])

        U, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=MAX_VIRTUAL_BOND_DIM,
            form='B'
        )

        wl = U.split_legs().replace_labels([ll, ll_hc], ['p', 'p*'])
        #wl.itranspose(['p', 'p*'])
        out_ws.append(wl)

        t = VH.split_legs()

    wl = t.replace_labels(
        [p_leg_labels[-1], p_leg_labels_hc[-1]],
        ['p', 'p*']
    )
    #wl.itranspose(['p', 'p*'])
    out_ws.append(wl)

    out_ws[0] = (
        out_ws[0]
        .add_leg(left_trivial_leg_charge, 0, label='vL')
    )
    
    out_ws[-1] = (
        out_ws[-1]
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out_ws

In [101]:
def split_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
            p_leg_labels=None):
    leg_label = get_physical_leg_labels(u)[0]

    if is_single_physical_leg_label(leg_label):
        return [unitary_to_mpo_tensor(u),]
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_u(u, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError
    

In [35]:
def unitary_to_mpo_tensor(u):
    out = (
        u
        .add_leg(left_trivial_leg_charge, 0, label='vL')
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out

In [36]:
def multiply_individual_mpo_tensors(w1, w2):
    w1 = w1.replace_labels(['vR', 'vL'], ['vR1', 'vL1'])
    w2 = w2.replace_labels(['vR', 'vL'], ['vR2', 'vL2'])

    w = (
        npc.tensordot(w1, w2, [['p'], ['p*']])
        .combine_legs([['vR1', 'vR2'], ['vL1', 'vL2']])
        .replace_labels(['(vR1.vR2)', '(vL1.vL2)'], ['vR', 'vL'])
    )

    return w

In [122]:
def group_elements(l, group_size, offset=0):
    first, rest = l[:offset], l[offset:]

    num_rest_groups = ((len(rest)-1)//group_size) + 1

    groups = [first,] if first else list()

    for i in range(num_rest_groups):
        first_index = i*group_size
        last_index = (i+1)*group_size
        groups.append(rest[first_index:last_index])

    return groups

In [38]:
def combine_grouped_b_tensors(grouped_bs):
    out = list()

    for group in grouped_bs:
        if len(group) == 1:
            out.append(group[0])
        else:
            out.append(combine_b_tensors(group))

    return out

In [104]:
def reduce_virtual_mpo_dim(wl, wr):
    # wl = wl.replace_labels(['p', 'p*'], ['pl', 'pl*'])

    wl = wl.combine_legs(['p', 'vL', 'p*'])
    wr = wr.combine_legs(['p', 'vR', 'p*'])
    w = npc.tensordot(wl, wr, ['vR', 'vL'])

    wl, wr = svd_reduce_split_tensor(
        w,
        max_inner_dim=MAX_VIRTUAL_BOND_DIM,
        form='B'
    )

    wl = wl.split_legs()
    wr = wr.split_legs()

    return wl, wr

In [111]:
def multiply_mpos(top_ws, bottom_ws):
    ws = list()
    
    for w1, w2 in zip(top_ws, bottom_ws):
        w = multiply_individual_mpo_tensors(w1, w2)
        ws.append(w)


    for i in range(len(ws)-1):
        ws[i], ws[i+1] = reduce_virtual_mpo_dim(ws[i], ws[i+1])

    return ws

# Optimisation

## First right sweep

In [39]:
test.right_symmetry_index

149

In [40]:
bs = [
    test_psi.get_B(i)
    for k, i in enumerate(range(test.right_symmetry_index + 1, test.right_symmetry_index + 12))
]

In [41]:
len(bs)

11

In [42]:
grouped_bs = group_elements(bs, 3, 0)

In [43]:
grouped_bs

[[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]]

In [44]:
combined_bs = combine_grouped_b_tensors(grouped_bs)

In [45]:
combined_bs

[<npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(p0.p1)', 'vR']>]

Again, the naming convention could be an issue.

In [49]:
unitaries = [
    get_identity_operator(t) for t in combined_bs
]

In [50]:
unitaries

[<npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(4, 4) labels=['(p0.p1)', '(p0*.p1*)']>]

In [51]:
combined_bs[1].conj()

<npc.Array shape=(8, 8, 8) labels=['vL*', '(p0*.p1*.p2*)', 'vR*']>

In [52]:
expectations = list()
expectations.append(list())

In [53]:
exps, *_ = one_site_optimization_sweep_right(
    test.right_projected_symmetry_state,
    combined_bs,
    unitaries
)

expectations[-1].append(exps)

In [54]:
expectations[-1]

[[1.1772577977037582,
  1.1786923957132358,
  1.1786923957132351,
  1.1786923957132365]]

In [55]:
new_combined_bs = list()

for b, u in zip(combined_bs, unitaries):
    ll = get_physical_leg_labels(b)[0]
    llh = conjugate_leg_label(ll)

    new_b = npc.tensordot(b, u, [[ll,], [llh,]])

    new_combined_bs.append(new_b)

In [56]:
new_combined_bs

[<npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(p0.p1)']>]

In [57]:
current_top_bs = list()

In [58]:
for b in new_combined_bs:
    current_top_bs.extend(split_combined_b(b))

In [59]:
current_top_bs

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

Test overlap

In [60]:
t = test.right_projected_symmetry_state

for bu, bd in zip(current_top_bs, bs):
    t = npc.tensordot(t, bu, [['vR',],['vL',]])
    t = npc.tensordot(t, bd.conj(), [['vR*', 'p'],['vL*', 'p*']])

e = npc.trace(t)

In [61]:
e

(1.175927790804166+0j)

Correct.

In [62]:
unitaries

[<npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(4, 4) labels=['(p0.p1)', '(p0*.p1*)']>]

In [63]:
unitaries[1].split_legs()

<npc.Array shape=(2, 2, 2, 2, 2, 2) labels=['p0', 'p1', 'p2', 'p0*', 'p1*', 'p2*']>

In [64]:
current_unitary_w_tensors = list()

for u in unitaries:
    current_unitary_w_tensors.extend(split_combined_u(u))

In [65]:
current_unitary_w_tensors

[<npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>]

## Second sweep

In [66]:
grouped_bs = group_elements(bs, 3, 1)
grouped_current_bs = group_elements(current_top_bs, 3, 1)

In [67]:
grouped_bs

[[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>],
 [<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]]

In [68]:
combined_bs = combine_grouped_b_tensors(grouped_bs)
combined_current_bs = combine_grouped_b_tensors(grouped_current_bs)

In [69]:
combined_bs

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

In [70]:
combined_current_bs

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

In [71]:
unitaries = [
    get_identity_operator(t) for t in combined_bs
]

In [72]:
unitaries

[<npc.Array shape=(2, 2) labels=['p', 'p*']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(2, 2) labels=['p', 'p*']>]

In [73]:
expectations.append(list())

In [74]:
expectations

[[[1.1772577977037582,
   1.1786923957132358,
   1.1786923957132351,
   1.1786923957132365]],
 []]

In [75]:
exps, *_ = one_site_optimization_sweep_right(
    test.right_projected_symmetry_state,
    combined_current_bs,
    unitaries,
    combined_bs
)

expectations[-1].append(exps)

In [76]:
expectations[-1]

[[1.175927790804165,
  1.303628725063367,
  1.3043109643048805,
  1.3043190341509328,
  1.3043190341509345]]

In [77]:
expectations

[[[1.1772577977037582,
   1.1786923957132358,
   1.1786923957132351,
   1.1786923957132365]],
 [[1.175927790804165,
   1.303628725063367,
   1.3043109643048805,
   1.3043190341509328,
   1.3043190341509345]]]

In [78]:
new_combined_bs = list()

for b, u in zip(combined_current_bs, unitaries):
    ll = get_physical_leg_labels(b)[0]
    llh = conjugate_leg_label(ll)

    new_b = npc.tensordot(b, u, [[ll,], [llh,]])

    new_combined_bs.append(new_b)

In [79]:
new_combined_bs

[<npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>]

In [87]:
current_top_bs = list()

In [88]:
for b in new_combined_bs:
    current_top_bs.extend(split_b(b))

In [89]:
current_top_bs

[<npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>]

Test overlap

In [90]:
t = test.right_projected_symmetry_state

for bu, bd in zip(current_top_bs, bs):
    t = npc.tensordot(t, bu, [['vR',],['vL',]])
    t = npc.tensordot(t, bd.conj(), [['vR*', 'p'],['vL*', 'p*']])

e = npc.trace(t)

In [91]:
e

(1.3022922268378696+0j)

In [92]:
unitaries

[<npc.Array shape=(2, 2) labels=['p', 'p*']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(2, 2) labels=['p', 'p*']>]

In [58]:
unitaries[1].split_legs()

<npc.Array shape=(2, 2, 2, 2) labels=['pl*', 'pr*', 'pl', 'pr']>

To do:
* Get w tensors
* Contract with prior w tensors.

In [93]:
current_unitary_w_tensors

[<npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>]

In [94]:
prior_unitary_w_tensors = current_unitary_w_tensors
current_unitary_w_tensors = list()

In [99]:
current_unitary_w_tensors = list()

for u in unitaries:
    current_unitary_w_tensors.extend(split_u(u))

In [100]:
current_unitary_w_tensors

[<npc.Array shape=(1, 1, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(2, 2, 4, 4) labels=['p', 'p*', 'vL', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 1, 2, 2) labels=['vR', 'vL', 'p', 'p*']>]

In [113]:
prior_unitary_w_tensors = multiply_mpos(prior_unitary_w_tensors, current_unitary_w_tensors)

## Third sweep

In [126]:
grouped_bs = group_elements(bs, 3, 2)
grouped_current_bs = group_elements(current_top_bs, 3, 2)

In [128]:
combined_bs = combine_grouped_b_tensors(grouped_bs)
combined_current_bs = combine_grouped_b_tensors(grouped_current_bs)

In [129]:
combined_bs

[<npc.Array shape=(8, 4, 8) labels=['vL', '(p0.p1)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>]

In [130]:
combined_current_bs

[<npc.Array shape=(8, 4, 8) labels=['vL', '(p0.p1)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', '(p0.p1.p2)', 'vR']>]

In [131]:
unitaries = [
    get_identity_operator(t) for t in combined_bs
]

In [132]:
unitaries

[<npc.Array shape=(4, 4) labels=['(p0.p1)', '(p0*.p1*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>,
 <npc.Array shape=(8, 8) labels=['(p0.p1.p2)', '(p0*.p1*.p2*)']>]

In [133]:
expectations.append(list())

In [134]:
expectations

[[[1.1772577977037582,
   1.1786923957132358,
   1.1786923957132351,
   1.1786923957132365]],
 [[1.175927790804165,
   1.303628725063367,
   1.3043109643048805,
   1.3043190341509328,
   1.3043190341509345]],
 []]

In [137]:
exps, *_ = one_site_optimization_sweep_right(
    test.right_projected_symmetry_state,
    combined_current_bs,
    unitaries,
    combined_bs
)

expectations[-1].append(exps)

In [138]:
expectations[-1]

[[1.302309102183223,
  1.3037161647937843,
  1.3055127771561226,
  1.3055152235921883],
 [1.305516414887383, 1.3055166447799396, 1.30551665006497, 1.305516650232087]]

In [139]:
expectations

[[[1.1772577977037582,
   1.1786923957132358,
   1.1786923957132351,
   1.1786923957132365]],
 [[1.175927790804165,
   1.303628725063367,
   1.3043109643048805,
   1.3043190341509328,
   1.3043190341509345]],
 [[1.302309102183223,
   1.3037161647937843,
   1.3055127771561226,
   1.3055152235921883],
  [1.305516414887383,
   1.3055166447799396,
   1.30551665006497,
   1.305516650232087]]]

In [140]:
new_combined_bs = list()

for b, u in zip(combined_current_bs, unitaries):
    ll = get_physical_leg_labels(b)[0]
    llh = conjugate_leg_label(ll)

    new_b = npc.tensordot(b, u, [[ll,], [llh,]])

    new_combined_bs.append(new_b)

In [141]:
new_combined_bs

[<npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(p0.p1)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>,
 <npc.Array shape=(8, 8, 8) labels=['vL', 'vR', '(p0.p1.p2)']>]

In [142]:
current_top_bs = list()

In [143]:
for b in new_combined_bs:
    current_top_bs.extend(split_b(b))

In [144]:
current_top_bs

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

Test overlap

In [145]:
t = test.right_projected_symmetry_state

for bu, bd in zip(current_top_bs, bs):
    t = npc.tensordot(t, bu, [['vR',],['vL',]])
    t = npc.tensordot(t, bd.conj(), [['vR*', 'p'],['vL*', 'p*']])

e = npc.trace(t)

In [146]:
e

(1.299239230931257+0j)

Likely have a name clash somewhere. Time to improve automation.